In [3]:
import tra_analysis as an
import numpy as np
import math
from multiprocessing import Pool
from functools import partial
from itertools import product

In [24]:
MAX_SCORE = 150
SCALING_FACTOR = 1

def min_score(n):
    return max(MAX_SCORE - 20*(n-1), 50)

def l_q(a):
    a_low = a[:math.ceil(len(a)/2)]
    return an.Analysis.median(a_low)
    #return np.percentile(a, 25)

def h_q(a):
    a_high = a[len(a)//2:]
    return an.Analysis.median(a_high)
    #return np.percentile(a, 75)

def r_lower(a, q1, q3):
    return q1 - SCALING_FACTOR*(q3 - q1)

def r_upper(a, q1, q3):
    return q3 + SCALING_FACTOR*(q3 - q1)

def bound(score, lower, upper):
    return max(min(upper, score), lower)

def b_first(a, time=False):
    if(time):
        return an.Analysis.npmin(a)
    else:
        return an.Analysis.npmax(a)

def b_last(a, time=False):
    if(time):
        return an.Analysis.npmax(a)
    else:
        return an.Analysis.npmin(a)

def compute_c(B, B_first, B_last, C_max, C_min):
    return abs((B - B_last) / (B_first - B_last))*(C_max - C_min) + C_min

def compute_score(score, min, max, R_L, R_U, B_F, B_L):
    BOUND = bound(score, R_L, R_U)
    return compute_c(BOUND, B_F, B_L, max, min)

def one_iter(array,time = False):

    MIN_SCORE = min_score(len(array))
    Q1 = l_q(array)
    Q3 = h_q(array)

    R_L = r_lower(array, Q1, Q3)
    R_U = r_upper(array, Q1, Q3)

    bounded_group = []
    for i in range(len(array)):
        bounded_group.append(bound(array[i], R_L, R_U))

    B_F = b_first(bounded_group, time)
    B_L = b_last(bounded_group, time)
    #new_exec_threads = Pool(processes = 8)
    results = map( partial (compute_score, min = MIN_SCORE, max = MAX_SCORE, R_L = R_L, R_U = R_U, B_F = B_F, B_L = B_L), array)
    return list(results)

In [25]:
sample_data = [10.0,25.0,37.1,38.2,49.3,53.0,56.1,59.5,70.5,120.5]
sample_data = an.Sort.quicksort(sample_data)

global exec_threads
exec_threads = Pool(processes = 8)

#compute_score(0, sample_data, bounded_data, time = True)

results = one_iter(sample_data, time = True)
results

[150.0,
 134.67261904761904,
 116.66666666666667,
 115.02976190476191,
 98.51190476190477,
 93.00595238095238,
 88.39285714285714,
 83.33333333333334,
 66.96428571428572,
 50.0]

In [26]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [48]:
range_low = 10
range_high = 11

num_teams = 6

score_set = np.arange(range_low*10, range_high*10+1)/10

big_set = []
for i in range(num_teams):
    big_set.append(score_set)

score_space = cartesian_product(*big_set)
#exec_threads.map(partial(one_iter, exec_threads = exec_threads), score_space)
#for i in range(len(score_space)-1):
    #print(score_space[i])
    #one_iter(score_space[i], exec_threads)
results = exec_threads.map(partial(one_iter, time = False), score_space)

In [49]:
results = np.array(results)
results[~np.isnan(results).any(axis=1)]

array([[ 50.,  50.,  50.,  50., 150., 150.],
       [ 50.,  50.,  50.,  50., 100., 150.],
       [ 50.,  50.,  50.,  50., 100., 150.],
       ...,
       [150., 100., 100., 150., 150.,  50.],
       [150.,  50.,  50., 150., 150.,  50.],
       [150.,  50.,  50., 150., 150., 150.]])